In [1]:
# import & 文件路径
import mne
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from Globals import *
from mne.preprocessing import ICA, create_eog_epochs
import csv
%matplotlib qt

In [2]:
os.chdir('/Volumes/Media/毕业论文/实验2数据/pre/Sub_013')
raw = mne.io.read_raw_brainvision('013.vhdr', eog = ['EOG'], preload = True)
raw.set_eeg_reference('average', ch_type = 'eeg')
raw.set_montage('standard_1020')

# 添加坏导
raw.info['bads'].append('AF8')
raw.info['bads'].append('P2')
raw.info['bads'].append('CPz')
raw.info['bads'].append('AF4')

#raw.info['bads']

# 插值坏导
raw.interpolate_bads()

# 将原始文件中的 annotation 读入 raw_annot 中；
raw_annot = raw.annotations
# make_relax_annotation 函数将 marker 之间间隔超过4.5秒的数据作为休息时间进行标记；
bad_relax_annot = make_relax_annotations(raw)
#将休息时间与原始的 annotation 合并；
raw.set_annotations(raw_annot + bad_relax_annot)
#crop_raw_relax函数对实验末尾的休息时间进行标记；
crop_raw_relax(raw)

raw.filter(0.1,40)
raw_for_ica = raw.copy().filter(1,40)

Extracting parameters from 013.vhdr...
Setting channel info structure...
Reading 0 ... 4388459  =      0.000 ...  4388.459 secs...
Applying average reference.
Applying a custom EEG reference.
Interpolating bad channels
    Automatic origin fit: head of radius 91.9 mm
Computing interpolation matrix from 59 sensor positions
Interpolating 4 sensors
Used Annotations descriptions: ['Comment/actiCAP Data On', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 15', 'Stimulus/S 16', 'Stimulus/S 17', 'Stimulus/S 18', 'Stimulus/S 19', 'Stimulus/S 22', 'Stimulus/S 33']
Used Annotations descriptions: ['Comment/actiCAP Data On', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 15', 'Stimulus/S 16', 'Stimulus/S 17', 'Stimulus/S 18', 'Stimulus/S 19', 'Stimulus/S 22', 'Stimulus

In [3]:
os.chdir('/Volumes/Media/毕业论文/实验2数据/pre/ica_fif')
ica = mne.preprocessing.read_ica('13_ica-ica.fif')
ica.plot_components()

Reading 13_ica-ica.fif ...
Now restoring ICA solution ...
Ready.


/var/folders/46/q563pnw55bjcvsn5bdg3h4t80000gn/T/ipykernel_32932/3630905085.py:2: DeprecationWarning: Version 0.23 introduced max_iter="auto", setting max_iter=1000 for `fastica` and max_iter=500 for `infomax` and `picard`. The current default of max_iter=200 will be changed to "auto" in version 0.24.
  ica = mne.preprocessing.read_ica('13_ica-ica.fif')


[<MNEFigure size 975x931 with 20 Axes>,
 <MNEFigure size 975x931 with 20 Axes>,
 <MNEFigure size 975x931 with 19 Axes>]

In [ ]:
ica.plot_properties(raw_for_ica,[0,1,2,3,13,14])

In [4]:
ica.exclude = [1,13]
ica.apply(raw)

Applying ICA to Raw instance
    Transforming to ICA space (59 components)
    Zeroing out 2 ICA components
    Projecting back using 63 PCA components


<RawBrainVision | 013.eeg, 64 x 4067858 (4067.9 s), ~1.94 GB, data loaded>

In [5]:
os.chdir('/Volumes/Media/毕业论文/实验2数据/pre/Sub_013')

# 从原始数据的 annotation 中读取 events(包含marker及对应时间点的数组) & event_id(marker名称列表)
events, event_id = mne.events_from_annotations(raw)

# get_corrected_events 函数删除 events 中错误反应以及错误反应对应的刺激marker；
corrected_events = get_corrected_events(events)
# 可视化校正后的 events;
mne.viz.plot_events(corrected_events)
# 对 events 进行切片；
event_1_third , event_2_third = crop_raw_1third_v2(corrected_events)
event_1_two, event_2_two = crop_raw_1two(corrected_events)

# 对切片后的raw数据进行分段；
epochs_1_third = mne.Epochs(raw, events= event_1_third,event_id = eventdict_1 , reject= dict(eeg = MAX_PEAK2PEAK), tmax= 0.5, tmin = -0.2, preload = True)
epochs_2_third = mne.Epochs(raw, events= event_2_third, event_id = eventdict_1 , reject= dict(eeg = MAX_PEAK2PEAK), tmax= 0.5, tmin = -0.2, preload = True)
epochs_1_two = mne.Epochs(raw, events= event_1_two, event_id = eventdict_1 , reject= dict(eeg = MAX_PEAK2PEAK), tmax= 0.5, tmin = -0.2, preload = True)
epochs_2_two = mne.Epochs(raw, events= event_2_two, event_id = eventdict_1 , reject= dict(eeg = MAX_PEAK2PEAK), tmax= 0.5, tmin = -0.2, preload = True)
# 完整数据分段
epochs_all = mne.Epochs(raw, events = corrected_events,event_id = eventdict_1,  reject= dict(eeg = MAX_PEAK2PEAK), tmax= 0.5, tmin = -0.2, preload = True )

epochs_1_third.save('1_3_epochs_epo.fif', overwrite = True)
epochs_2_third.save('2_3_epochs_epo.fif', overwrite = True)
epochs_1_two.save('1_2_epochs_epo.fif', overwrite = True)
epochs_2_two.save('2_2_epochs_epo.fif', overwrite = True)
epochs_all.save('epochs_all_epo.fif', overwrite = True)

Used Annotations descriptions: ['Comment/actiCAP Data On', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 15', 'Stimulus/S 16', 'Stimulus/S 17', 'Stimulus/S 18', 'Stimulus/S 19', 'Stimulus/S 22', 'Stimulus/S 33']


/var/folders/46/q563pnw55bjcvsn5bdg3h4t80000gn/T/ipykernel_32932/806771110.py:9: RuntimeWarning: More events than default colors available. You should pass a list of unique colors.
  mne.viz.plot_events(corrected_events)


Not setting metadata
Not setting metadata
273 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 273 events and 701 original time points ...
    Rejecting  epoch based on EEG : ['TP9']
    Rejecting  epoch based on EEG : ['TP9']
    Rejecting  epoch based on EEG : ['TP9']
    Rejecting  epoch based on EEG : ['FT9', 'TP9', 'FT10']
    Rejecting  epoch based on EEG : ['TP9']
    Rejecting  epoch based on EEG : ['TP9']
    Rejecting  epoch based on EEG : ['TP9', 'T8', 'FT10', 'FT8']
    Rejecting  epoch based on EEG : ['FT9', 'TP9', 'T8', 'FT10', 'AF7', 'FT7', 'FT8']
    Rejecting  epoch based on EEG : ['TP9']
    Rejecting  epoch based on EEG : ['TP9', 'T8', 'FT10', 'FT7', 'FT8']
    Rejecting  epoch based on EEG : ['FT9', 'TP9', 'T8', 'FT10', 'AF3', 'F5', 'FT7', 'FT8']
    Rejecting  epoch based on EEG : ['TP9']
    Rejecting  epoch based on EEG : ['TP9', 'FT7']
    Rejecting  epoch b